# Create and store features

feature engineering can be push to the next level when feature became a real artefact, instead of getting flat parquet columns, we will query the feature store here called FEAST to get the real value of a feature

In [1]:
#!pip install feast
import pandas as pd
### Feast dependancies
from feast import (
    FeatureStore,
    Entity,
    FeatureService,
    FeatureView,
    Field,
    FileSource,
    PushSource,
    RequestSource,
    RepoConfig
)
from feast.on_demand_feature_view import on_demand_feature_view
from feast.types import Float32, Float64, Int64,UnixTimestamp
import requests
import json
from datetime import timedelta, datetime

/opt/conda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


### Level 0 : feature store without modelisation

The feature store take its advantage of central entities definition and global reusable features, In this part, we will just adress the feature store as a column store that we will reuse for training a model

#### Initialize a repo

In [2]:
fs = FeatureStore(repo_path="./feature_repo")

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Ensure data requirements

In [3]:
## read local data from local repository
data = ...

In [4]:
### Transform trip_start_timestamp field to a datetime field
data[['trip_start_timestamp']] = ...
### add an index column that will be used as a join key
data = ...

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Parquet format

In [5]:
### Persist locally as parquet data
data...

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Create an Entity

In [6]:
# fill with your initials --> john-doe initials : jd
username_initials = ...

In [7]:
### Entity is a feature "catalogue", it can be viewed as a mapping between a business concept and some training features
taxi_trip_entity = Entity(name=f"taxi_trip_{username_initials}", join_keys=["index"])


### Create a source 

In [8]:
### File source is used when we materialize features
## we will use our local data in parquet
## the event timestamps column will be the only in timestamp in current data
taxi_data_source = FileSource(
    path=...,
        event_timestamp_column=...

)

### Create a feature view

In [9]:
### Now we will create a feature view where we will map our entity with the source
### fill the schema with the dataframe schema and using feast imported dtypes
### the source is the one we declared earlier
taxi_trip_global_view = FeatureView(
    name=f"taxi_trip_global_view_{username_initials}",
    ttl=timedelta(seconds=31536000 * 3),
    entities=[taxi_trip_entity],
    schema=[
        Field(name="index", dtype=Int64),
        ...
        ],    
    source=...
)

### Apply this model to our project

In [10]:
### we want to apply to objects we created (entities, features views)
fs.apply([...,...])

/opt/conda/lib/python3.8/site-packages/feast/infra/offline_stores/file_source.py:161: FutureWarning: 'ParquetDataset.schema' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.schema' attribute instead (which will return an Arrow schema instead of a Parquet schema).
  schema = ParquetDataset(path).schema.to_arrow_schema()


Now in the FEAST UI, we can see the model that we just created, browse the feature view to see what has been created

![entity](images/entity.png)

### Materialization

Now we will use our local data and feed it to the taxi_trip_{username_initials}

In [11]:
fs.materialize_incremental(end_date=datetime.now())

Materializing 1 feature views to 2023-01-10 11:12:02+00:00 into the postgres online store.

taxi_trip_global_view_ge from 2020-01-11 11:12:02+00:00 to 2023-01-10 11:12:02+00:00:


/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:1722: FutureWarning: 'ParquetDataset.partitions' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.partitioning' attribute instead.
  if dataset.partitions is not None:
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:1750: FutureWarning: 'ParquetDataset.metadata' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version.
  if dataset.metadata:
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:1769: FutureWarning: 'ParquetDataset.schema' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.schema' attribute instead (which will return an Arrow schema instead of a Parquet schema).
  if dataset